<a href="https://colab.research.google.com/github/kravipa1/cakechat/blob/master/kohya_ss_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Train with Kohya's Stable Diffusion Trainers
%cd /content

from google.colab import drive
drive.mount('/content/drive')

!pip install dadaptation==3.1 diffusers[torch]==0.17.1 easygui==0.98.3 einops==0.6.0 fairscale==0.4.13 ftfy==6.1.1 gradio==3.36.1
!pip install lion-pytorch==0.0.6 lycoris_lora==1.8.0.dev6 open-clip-torch==2.20.0 prodigyopt==1.0 pytorch-lightning==1.9.0 safetensors==0.3.1 timm==0.6.12
!pip install tk==0.1.0 transformers==4.30.2 voluptuous==0.13.1 wandb==0.15.0 xformers==0.0.20 omegaconf
!pip install torchvision==0.15.2 huggingface_hub==0.16.4 accelerate==0.21.0 --force-reinstall


%cd /content
!git clone -b 0.41.0 https://github.com/TimDettmers/bitsandbytes
%cd /content/bitsandbytes
!CUDA_VERSION=118 make cuda11x
!python setup.py install

%cd /content
!git clone -b v1.0 https://github.com/camenduru/kohya_ss
%cd /content/kohya_ss

!python kohya_gui.py --share --headless

/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.1
    Uninstalling numpy-2.3.1:
      Successfully uninstalled numpy-2.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
sentence-transformers 4.1.0 requires huggingface-hub>=0.20.0, but you have huggingface-hub 0.17.1 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.30.2 which is incompatible.


  Using cached torchvision-0.15.2-cp311-cp311-manylinux1_x86_64.whl.metadata (11 kB)
  Using cached numpy-2.3.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached torch-2.0.1-cp311-cp311-manylinux1_x86_64.whl.metadata (24 kB)
  Using cached pillow-11.3.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (9.0 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-man

/content
fatal: destination path 'bitsandbytes' already exists and is not an empty directory.
/content/bitsandbytes
ENVIRONMENT
CUDA_VERSION: 118
NVCC path: /usr/local/cuda/bin/nvcc
GPP path: /usr/bin/g++ VERSION: g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
CUDA_HOME: /usr/local/cuda
CONDA_PREFIX: 
PATH: /opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
/usr/local/cuda/bin/nvcc -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -Xcompiler '-fPIC' --use_fast_math -Xptxas=-v -dc /content/bitsandbytes/csrc/ops.cu /content/bitsandbytes/csrc/kernels.cu -I /usr/local/cuda/include -I /content/bitsandbytes/csrc -I /include -I /content/bitsandbytes/include -L /usr/local/cuda/lib64 -lcudart -lcublas -lcublasLt -lcusparse -L /lib --output-directory /content/bitsandbytes/build


In [ ]:
#@title Convert Safetensors to Diffusers
from_safetensors_url = '' #@param {type:"string"}
!wget -q https://raw.githubusercontent.com/huggingface/diffusers/v0.17.1/scripts/convert_original_stable_diffusion_to_diffusers.py
!wget {from_safetensors_url} -O /content/model.safetensors
!python3 convert_original_stable_diffusion_to_diffusers.py --half --from_safetensors --checkpoint_path model.safetensors --dump_path /content/model

In [ ]:
#@title Push to HF.co

import os
from huggingface_hub import create_repo, upload_folder

hf_token = 'HF_WRITE_TOKEN' #@param {type:"string"}
repo_id = 'username/reponame' #@param {type:"string"}
commit_message = '\u2764' #@param {type:"string"}
create_repo(repo_id, private=True, token=hf_token)
model_path = '/content/train/model' #@param {type:"string"}
upload_folder(folder_path=f'{model_path}', path_in_repo='', repo_id=repo_id, commit_message=commit_message, token=hf_token)

In [ ]:
#@title Push to DagsHub.com

!pip -q install dagshub
from dagshub.upload import Repo, create_repo

repo_id = 'reponame' #@param {type:"string"}
org_name = 'orgname' #@param {type:"string"}
commit_message = '\u2764' #@param {type:"string"}
create_repo(f"{repo_id}", org_name=f"{org_name}")
repo = Repo(f"{org_name}", f"{repo_id}")
model_path = '/content/train/model' #@param {type:"string"}
repo.upload("/content/models", remote_path="data", commit_message=f"{commit_message}", force=True)